In [9]:
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

class YourCNN(nn.Module):
    def __init__(self):
        super(YourCNN, self).__init__()
        
        # Define the convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Define the dense layers
        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 39)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.dense_layers(x)
        return x

def predict_single_image(image_path):
    # Open the image using PIL
    image = Image.open(image_path)
    
    # Preprocess the image
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image_tensor = preprocess(image)
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
    
    # Perform inference
    with torch.no_grad():
        output = model(image_tensor)
        predicted_index = torch.argmax(output).item()
    
    return predicted_index

# Path to the pretrained model file
model_path = r'C:/Users/sneha/numpy tutorials/PLANT_DISEASE_MODEL/plant_disease_model_1_latest.pt'
csv_path = r'C:/Users/sneha/numpy tutorials/PLANT_DISEASE_MODEL/disease_info.csv' 

# Load the model
model = YourCNN()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()
disease_info_df = pd.read_csv(csv_path, encoding='latin-1') 

def get_disease_information(prediction_index):
    row = disease_info_df.loc[disease_info_df['index'] == prediction_index]
    if not row.empty:
        return row.to_dict(orient='records')[0]
    else:
        return {"name": "Unknown", "description": "No information available", "steps": "", "image_url": ""}

# Test the model on a single image
image_path = r'C:/Users/sneha/numpy tutorials/PLANT_DISEASE_MODEL/test_images/car_test.jpeg'  # Replace with the path to your test image
prediction_index = predict_single_image(image_path)

# Print the predicted index
disease_info = get_disease_information(prediction_index)

# Print the predicted disease information
print("Predicted Disease Name:", disease_info["disease_name"])
print("Predicted Disease Description:", disease_info["description"])
print("Possible Steps:", disease_info["Possible Steps"])
print("Image URL:", disease_info["image_url"])


Predicted Disease Name: Background Without Leaves
Predicted Disease Description: There is no leaf in the given image
Possible Steps: please reupload image with leaf
Image URL: https://cdn.onlinewebfonts.com/svg/img_332817.png
